# Clean the original Steam dataset (Kaggle)
*https://www.kaggle.com/datasets/tamber/steam-video-games*

In [103]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [130]:
df = pd.read_csv('steam-200k.csv')

In [131]:
df.head()

,151603712,The Elder Scrolls V Skyrim,purchase,1.0,0
0,151603712,The Elder Scrolls V Skyrim,play,273.0,0
1,151603712,Fallout 4,purchase,1.0,0
2,151603712,Fallout 4,play,87.0,0
3,151603712,Spore,purchase,1.0,0
4,151603712,Spore,play,14.9,0


In [132]:
df.shape

(199999, 5)

In [133]:
# rename the columns
cols = {'151603712':'user_id', 'The Elder Scrolls V Skyrim': 'name', 'purchase':'action', '1.0':'quantity'}
df.rename(columns = cols, inplace = True)

# get rid of useless 5th column (no information all zeros)
df.drop(df.columns[[4]], inplace=True, axis=1)
df

,user_id,name,action,quantity
0,151603712,The Elder Scrolls V Skyrim,play,273.0
1,151603712,Fallout 4,purchase,1.0
2,151603712,Fallout 4,play,87.0
3,151603712,Spore,purchase,1.0
4,151603712,Spore,play,14.9
...,...,...,...,...
199994,128470551,Titan Souls,play,1.5
199995,128470551,Grand Theft Auto Vice City,purchase,1.0
199996,128470551,Grand Theft Auto Vice City,play,1.5
199997,128470551,RUSH,purchase,1.0


In [134]:
df.head()

,user_id,name,action,quantity
0,151603712,The Elder Scrolls V Skyrim,play,273.0
1,151603712,Fallout 4,purchase,1.0
2,151603712,Fallout 4,play,87.0
3,151603712,Spore,purchase,1.0
4,151603712,Spore,play,14.9


In [135]:
# create a new row (missing Skyrim purchase) and concat at the front of df
new_row = pd.DataFrame({'user_id':151603712, 'name':'The Elder Scrolls V Skyrim', 
                        'action':'purchase', 'quantity':1}, index =[0])
df = pd.concat([new_row, df[:]]).reset_index(drop = True)
df.head()

,user_id,name,action,quantity
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0
1,151603712,The Elder Scrolls V Skyrim,play,273.0
2,151603712,Fallout 4,purchase,1.0
3,151603712,Fallout 4,play,87.0
4,151603712,Spore,purchase,1.0


In [136]:
df['name'].nunique()

5155

In [137]:
games_histg = df['name'].value_counts()
games_histg

Dota 2                             9682
Team Fortress 2                    4646
Counter-Strike Global Offensive    2789
Unturned                           2632
Left 4 Dead 2                      1752
                                   ... 
Putt-Putt Joins the Parade            1
Ducati World Championship             1
Chunk of Change Knight                1
STASIS                                1
Soccertron                            1
Name: name, Length: 5155, dtype: int64

In [138]:
# see which games are the "popular" ones (with >10 owners)
pop_games_m10 = df['name'].value_counts()[df['name'].value_counts() > 10]
# pop_games_m10.to_csv('pop_games.csv')
pop_games_m10

Dota 2                             9682
Team Fortress 2                    4646
Counter-Strike Global Offensive    2789
Unturned                           2632
Left 4 Dead 2                      1752
                                   ... 
The Maw                              11
Air Conflicts Pacific Carriers       11
X-Tension                            11
Dogfight 1942                        11
Sam & Max 105 Reality 2.0            11
Name: name, Length: 2099, dtype: int64

In [139]:
# how many unique users? 12393
df['user_id'].nunique()

12393

In [140]:
# how many purchase actions (129511), how many play actions (70489)
df1 = df['action'].value_counts()
print(df1)

purchase    129511
play         70489
Name: action, dtype: int64


In [141]:
# what are the "unpopular" games (with <= 10 owners)
unpop_games_u10 = df['name'].value_counts()[df['name'].value_counts() <= 10]

unpop_games_u10 = unpop_games_u10.rename_axis("name").reset_index(name='counts')

unpop_games_u10

,name,counts
0,Space Empires IV Deluxe,10
1,Contrast,10
2,Cities XXL,10
3,Fortix,10
4,Out of the Park Baseball 15,10
...,...,...
3051,Putt-Putt Joins the Parade,1
3052,Ducati World Championship,1
3053,Chunk of Change Knight,1
3054,STASIS,1


In [142]:
# how many with purchase == 1? Same as number of purchases
len(df[(df['action'] == 'purchase') & 
             (df['quantity'] == 1)])

129511

In [143]:
# how many with purchase == 0? None, sanity check make sure games listed are all purchased
len(df[(df['action'] == 'purchase') & 
             (df['quantity'] == 0)])

0

In [144]:
# create a new df2 that removes all the unpopular games (<11 owners)
df2 = df[~df['name'].isin(unpop_games_u10['name'])]

In [145]:
# make a histogram of owner counts for each game
games_histg2 = df2['name'].value_counts()
games_histg2

Dota 2                                      9682
Team Fortress 2                             4646
Counter-Strike Global Offensive             2789
Unturned                                    2632
Left 4 Dead 2                               1752
                                            ... 
Crusader Kings II Hymns of Abraham            11
Europa Universalis IV American Dream DLC      11
Her Story                                     11
Black Fire                                    11
Jagged Alliance Online - Steam Edition        11
Name: name, Length: 2099, dtype: int64

In [146]:
# how many unique games there are after filtering out the unpopular games (<11 owners)
df2['name'].nunique()

2099

In [147]:
# how many rows are left from the original 199,999 after removing the unpopular games (<11 owners)
df2.shape

(188781, 4)

In [148]:
# output the "popular" games in the user action dataset to csv (removed unpop games)
# df2.to_csv("temp/steam_pop.csv")

In [149]:
# how many unique users are left after removing the unpopular game users
df2["user_id"].nunique()

12299

In [150]:
# pivot the table so that play and purchase are separate columns
df3 = df2.pivot_table(index=['user_id','name'], columns='action', values='quantity', fill_value=0)
df3

action                                   play  purchase
user_id   name                                         
5250      Alien Swarm                     4.9         1
          Cities Skylines               144.0         1
          Counter-Strike                  0.0         1
          Counter-Strike Source           0.0         1
          Day of Defeat                   0.0         1
...                                       ...       ...
309626088 Age of Empires II HD Edition    6.7         1
309812026 Counter-Strike Nexon Zombies    0.0         1
          Robocraft                       0.0         1
309824202 Dota 2                          0.7         1
309903146 Dota 2                          0.2         1

[120407 rows x 2 columns]

In [151]:
# output the "popular" games in the pivoted user action dataset to csv (rm unpop games)
df3.to_csv('temp/steam_pivoted.csv')

In [152]:
# read back to df4, alternative way of flattening the multi-index table
df4 = pd.read_csv('temp/steam_pivoted.csv')

In [153]:
df4

,user_id,name,play,purchase
0,5250,Alien Swarm,4.9,1
1,5250,Cities Skylines,144.0,1
2,5250,Counter-Strike,0.0,1
3,5250,Counter-Strike Source,0.0,1
4,5250,Day of Defeat,0.0,1
...,...,...,...,...
120402,309626088,Age of Empires II HD Edition,6.7,1
120403,309812026,Counter-Strike Nexon Zombies,0.0,1
120404,309812026,Robocraft,0.0,1
120405,309824202,Dota 2,0.7,1


In [154]:
# get dataframe of the 2099 unique games in the dataset, turn into csv to augment features
unique_games = df4['name'].unique()
unique_games_df = pd.DataFrame(unique_games, columns = ['name'])

In [155]:
# read in the Steam game library metadata dataset (release_date, categories, genre, price, etc)
lib_df = pd.read_csv('archive/steam.csv')
lib_df.head()

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99


In [156]:
# Copy the game name to og_name column to retain the original spelling/punctuation 
# lib_df['og_name'] = lib_df['name']

# Move og_name column to the 3rd column behind name
# og_col = lib_df.pop('og_name')
# lib_df.insert(2, 'og_name', og_col)

# get rid of special char and whitespaces in the name for the library df (steam mine repo)
lib_df['name'] = lib_df.name.replace('[^a-zA-Z0-9]+', '', regex=True)

# convert the names to lowercase
lib_df['name'] = lib_df['name'].str.lower()

In [157]:
# output game library metadata to lib_df.csv
lib_df.to_csv('temp/lib_df_temp.csv')

# special condition get rid of the unpopular 'dwarfs', 'home', 'rush', 'escape' games 
# (same name as the popular ones -diff punctuation) 
# if left in, the each instance of 'dwarfs','home',etc in the user action dataset will have another row joined to 
# it, inflating the dataset
lib_df = lib_df[lib_df.appid != 281560]
lib_df = lib_df[lib_df.appid != 1051170] 
lib_df = lib_df[lib_df.appid != 844040] 
lib_df = lib_df[lib_df.appid != 759010] 
lib_df = lib_df[lib_df.appid != 806600]

# output game library metadata to lib_df.csv
lib_df.to_csv('temp/lib_df.csv')

In [158]:
# Copy the game name to og_name column to retain the original spelling/punctuation 
unique_games_df['og_name'] = unique_games_df['name']

# Move og_name column to the 3rd column behind name
og_col2 = unique_games_df.pop('og_name')
unique_games_df.insert(1, 'og_name', og_col2)

# get rid of special char and whitespaces in the unique games df (user action repo)
unique_games_df['name'] = unique_games_df.name.replace('[^a-zA-Z0-9]+', '', regex=True)

# convert the names to lowercase
unique_games_df['name'] = unique_games_df['name'].str.lower()

In [159]:
unique_games_df.value_counts()

name                   og_name                
100orangejuice         100% Orange Juice          1
ricochet               Ricochet                   1
reversiontheescape     Reversion - The Escape     1
revengeofthetitans     Revenge of the Titans      1
reus                   Reus                       1
                                                 ..
finalfantasyxiii       FINAL FANTASY XIII         1
finalfantasyviii       FINAL FANTASY VIII         1
finalfantasyvii        FINAL FANTASY VII          1
finalfantasyiii        FINAL FANTASY III          1
zombiesmonstersrobots  Zombies Monsters Robots    1
Length: 2099, dtype: int64

In [160]:
unique_games_df['name'].nunique()

2097

In [161]:
# remove the bug where ruse and universesandbox appears twice in the unique games df
#unique_games_df = unique_games_df.drop_duplicates()
unique_games_df = unique_games_df.drop([1088,1577])
unique_games_df.value_counts()

name                       og_name                     
100orangejuice             100% Orange Juice               1
revolutionace              Revolution Ace                  1
revengeofthetitans         Revenge of the Titans           1
reus                       Reus                            1
returntocastlewolfenstein  Return to Castle Wolfenstein    1
                                                          ..
finalfantasyxiii           FINAL FANTASY XIII              1
finalfantasyviii           FINAL FANTASY VIII              1
finalfantasyvii            FINAL FANTASY VII               1
finalfantasyiii            FINAL FANTASY III               1
zombiesmonstersrobots      Zombies Monsters Robots         1
Length: 2097, dtype: int64

In [162]:
# output the unique game names (user action repo), check if all lowercase/no whitespace
unique_games_df.to_csv('temp/unique_games.csv')
unique_games_df

,name,og_name
0,alienswarm,Alien Swarm
1,citiesskylines,Cities Skylines
2,counterstrike,Counter-Strike
3,counterstrikesource,Counter-Strike Source
4,dayofdefeat,Day of Defeat
...,...,...
2094,fivenightsatfreddys3,Five Nights at Freddy's 3
2095,victimofxen,Victim of Xen
2096,piercingblow,Piercing Blow
2097,metalwaronlineretribution,Metal War Online Retribution


In [163]:
# check metadata library game name are lowercase/no whitespace (since joining on 'name')
lib_df

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,counterstrike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
1,20,teamfortressclassic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
2,30,dayofdefeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
3,40,deathmatchclassic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
4,50,halflifeopposingforce,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27070,1065230,roomofpandora,2019-04-24,1,SHEN JIAWEI,SHEN JIAWEI,windows,0,Single-player;Steam Achievements,Adventure;Casual;Indie,Adventure;Indie;Casual,7,3,0,0,0,0-20000,2.09
27071,1065570,cybergun,2019-04-23,1,Semyon Maximov,BekkerDev Studio,windows,0,Single-player,Action;Adventure;Indie,Action;Indie;Adventure,0,8,1,0,0,0-20000,1.69
27072,1065650,superstarblast,2019-04-24,1,EntwicklerX,EntwicklerX,windows,0,Single-player;Multi-player;Co-op;Shared/Split ...,Action;Casual;Indie,Action;Indie;Casual,24,0,1,0,0,0-20000,3.99
27073,1066700,newyankee7deerhunters,2019-04-17,1,Yustas Game Studio,Alawar Entertainment,windows;mac,0,Single-player;Steam Cloud,Adventure;Casual;Indie,Indie;Casual;Adventure,0,2,0,0,0,0-20000,5.19


In [164]:
lib_df['name'].value_counts()

                                                        92
2                                                        3
romanceofthethreekingdoms                                3
crush                                                    3
invasion                                                 3
                                                        ..
oddplanet                                                1
echoesofthepasttherevengeofthewitchcollectorsedition     1
darktalesedgarallanpoesthegoldbugcollectorsedition       1
darkparablesthefinalcinderellacollectorsedition          1
runelord                                                 1
Name: name, Length: 26878, dtype: int64

In [165]:
# join the unique game names dataset (user action repo) with metadata (library repo) on 'name'
merged_df = unique_games_df.merge(lib_df, on='name', how='left')
merged_df

,name,og_name,appid,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,alienswarm,Alien Swarm,630.0,2010-07-19,1.0,Valve,Valve,windows,0.0,Single-player;Multi-player;Co-op;Steam Achieve...,Action,Free to Play;Co-op;Action,66.0,17435.0,941.0,371.0,83.0,2000000-5000000,0.00
1,citiesskylines,Cities Skylines,255710.0,2015-03-10,1.0,Colossal Order Ltd.,Paradox Interactive,windows;mac;linux,0.0,Single-player;Steam Achievements;Steam Trading...,Simulation;Strategy,City Builder;Simulation;Building,97.0,67553.0,6005.0,3225.0,444.0,5000000-10000000,22.99
2,counterstrike,Counter-Strike,10.0,2000-11-01,1.0,Valve,Valve,windows;mac;linux,0.0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0.0,124534.0,3339.0,17612.0,317.0,10000000-20000000,7.19
3,counterstrikesource,Counter-Strike Source,240.0,2004-11-01,1.0,Valve,Valve,windows;mac;linux,0.0,Multi-player;Cross-Platform Multiplayer;Steam ...,Action,Action;FPS;Multiplayer,147.0,76640.0,3497.0,6842.0,400.0,10000000-20000000,7.19
4,dayofdefeat,Day of Defeat,30.0,2003-05-01,1.0,Valve,Valve,windows;mac;linux,0.0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0.0,3416.0,398.0,187.0,34.0,5000000-10000000,3.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2092,fivenightsatfreddys3,Five Nights at Freddy's 3,354140.0,2015-03-02,1.0,Scott Cawthon,Scott Cawthon,windows,0.0,Single-player,Action;Indie;Simulation,Horror;Singleplayer;Survival Horror,0.0,4917.0,678.0,133.0,146.0,200000-500000,5.79
2093,victimofxen,Victim of Xen,300220.0,2014-05-21,1.0,Smolders,Smolders,windows,0.0,Single-player;Steam Achievements;Full controll...,Adventure;Casual;RPG,RPGMaker;RPG;Adventure,7.0,265.0,163.0,223.0,308.0,100000-200000,2.99
2094,piercingblow,Piercing Blow,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2095,metalwaronlineretribution,Metal War Online Retribution,412470.0,2015-12-01,1.0,GDT Limited,GDT Limited,windows,0.0,Multi-player,Action;Free to Play;Massively Multiplayer;Raci...,Early Access;Free to Play;Multiplayer,0.0,438.0,361.0,76.0,78.0,200000-500000,0.00


In [166]:
# output to csv -- this is the unique games list (2104 games) but some with na info
merged_df.to_csv('temp/games_list_w_na.csv')

In [167]:
# count how many games are deprecated (has na info) from the current Steam library: 734
nan_count = merged_df['appid'].isna().sum()
nan_count

734

In [168]:
# output the game titles that are deprecated on Steam
missing_df = merged_df[merged_df['appid'].isnull()]
missing_df.to_csv('temp/missing.csv')

In [169]:
# get rid of special char and whitespaces in the unique games df (user action repo)
df4['name'] = df4.name.replace('[^a-zA-Z0-9]+', '', regex=True)

# convert the names to lowercase
df4['name'] = df4['name'].str.lower()

df4

,user_id,name,play,purchase
0,5250,alienswarm,4.9,1
1,5250,citiesskylines,144.0,1
2,5250,counterstrike,0.0,1
3,5250,counterstrikesource,0.0,1
4,5250,dayofdefeat,0.0,1
...,...,...,...,...
120402,309626088,ageofempiresiihdedition,6.7,1
120403,309812026,counterstrikenexonzombies,0.0,1
120404,309812026,robocraft,0.0,1
120405,309824202,dota2,0.7,1


In [170]:
# merge the user action dataset with the game meta data dataset
final_df = df4.merge(merged_df, on='name', how='left')
final_df.to_csv('temp/user_steam_lib_w_na.csv')

In [171]:
# drop any rows with na in the dataset (steam since dropped those games from store)
final_df = final_df.dropna()
final_df.to_csv('final2/user_steam_lib_final.csv')

In [172]:
# generate the unique game list, dropping the games that are deprecated (with na)
final_games_df = merged_df.dropna()
final_games_df.to_csv('final2/games_list_final.csv')

In [173]:
print(final_df['release_date'].min())
print(final_df['release_date'].max())

1998-11-08
2019-02-28
